# Module 2 Assessment

Welcome to your Mod 2 Assessment. You will be tested for your understanding of concepts and ability to solve problems that have been covered in class and in the curriculum.

Use any libraries you want to solve the problems in the assessment.

You will have up to two hours to complete this assessment.

The sections of the assessment are:

- Accessing Data Through APIs
- Object Oriented Programming
- SQL and Relational Databases
- HTML, CSS and Web Scraping
- Other Database Structures (MongoDB)

In this assessment you will be exploring two datasets: Pokemon and Quotes.

In [3]:
!pip install pymongo

    100% |████████████████████████████████| 317kB 3.9MB/s ta 0:00:01
You are using pip version 10.0.1, however version 19.2.1 is available.
You should consider upgrading via the 'pip install --upgrade pip' command.


In [4]:
# import the necessary libraries

import requests
import json
import pandas as pd
import sqlite3
from bs4 import BeautifulSoup
import pymongo

## Part 1: Accessing Data Through APIs

In this section we'll be using PokeAPI to get data on Pokemon. Let's first define functions to get information from the API. Provided below is a URL that will get you started with the first 151 Pokemon! Run the cell below to see what we get.

In [5]:
url = 'https://pokeapi.co/api/v2/pokemon/?limit=151'
results = requests.get(url).json()['results']
results

[{'name': 'bulbasaur', 'url': 'https://pokeapi.co/api/v2/pokemon/1/'},
 {'name': 'ivysaur', 'url': 'https://pokeapi.co/api/v2/pokemon/2/'},
 {'name': 'venusaur', 'url': 'https://pokeapi.co/api/v2/pokemon/3/'},
 {'name': 'charmander', 'url': 'https://pokeapi.co/api/v2/pokemon/4/'},
 {'name': 'charmeleon', 'url': 'https://pokeapi.co/api/v2/pokemon/5/'},
 {'name': 'charizard', 'url': 'https://pokeapi.co/api/v2/pokemon/6/'},
 {'name': 'squirtle', 'url': 'https://pokeapi.co/api/v2/pokemon/7/'},
 {'name': 'wartortle', 'url': 'https://pokeapi.co/api/v2/pokemon/8/'},
 {'name': 'blastoise', 'url': 'https://pokeapi.co/api/v2/pokemon/9/'},
 {'name': 'caterpie', 'url': 'https://pokeapi.co/api/v2/pokemon/10/'},
 {'name': 'metapod', 'url': 'https://pokeapi.co/api/v2/pokemon/11/'},
 {'name': 'butterfree', 'url': 'https://pokeapi.co/api/v2/pokemon/12/'},
 {'name': 'weedle', 'url': 'https://pokeapi.co/api/v2/pokemon/13/'},
 {'name': 'kakuna', 'url': 'https://pokeapi.co/api/v2/pokemon/14/'},
 {'name': '

[Read the documentation here](https://pokeapi.co/) for information on navigating this API and use the API to obtain data to answer the following questions.

### Accessing Data

1. For any **one** Pokemon, retrieve the following information in a dictionary format with the following keys:
    - ID
    - Name
    - Base experience
    - Weight
    - Height
    - Types
    - Abilities

For `Types` and `Abilities`, you might want to write helper functions to have each attribute just be a list of types and a list of abilities. Your output should look like this:

```
{'id': 1, 
'name': 'bulbasaur', 
'base_experience': 64, 
'weight': 69, 
'height': 7, 
'types': ['poison', 'grass'], 
'abilities': ['chlorophyll', 'overgrow']}

```
    


In [147]:
def get_pokedata_by_id(poke_id):
    """get_poke_data_by_id(id):
    Fetch individual pokemon data from pokeapi using id
    Params:
        id: known id of individual pokemon
    Returns:
        Pokeapi response data for individual pokemon
    """
    BASE_URL = 'https://pokeapi.co/api/v2/pokemon/{}'
    url = BASE_URL.format(poke_id)
    return requests.get(url).json()

In [148]:
def get_pokemon_dict(poke_result_dict):
    """get_pookemon_data(poke_result_dict):
    Pull out specific pokemon attributes from
    our result set and populate a new dictionary
    Params:
        poke_result_dict: full dict of pokemon 
        results from API
    Returns:
        dict of pokemon attributes
    """
    id = poke_result_dict['id']
    name = poke_result_dict['name']
    base_experience = poke_result_dict['base_experience']
    weight = poke_result_dict['weight']
    height = poke_result_dict['height']
    types = [typ['type']['name'] for typ in poke_result_dict['types'] ]
    abilities = [abil['ability']['name'] for abil in poke_result_dict['abilities'] ]
    return dict(id=id, name=name, 
                base_experience=base_experience, 
                weight=weight, 
                height=height, 
                types=types, 
                abilities=abilities)

In [149]:
# 
# Test our functions on test data:  use pokemon id=72 (tentacool)
#

In [150]:
# Get our data:
poke_response = get_pokedata_by_id('72')
poke_dict = get_pokemon_dict(poke_response)
poke_dict

{'id': 72,
 'name': 'tentacool',
 'base_experience': 67,
 'weight': 455,
 'height': 9,
 'types': ['poison', 'water'],
 'abilities': ['rain-dish', 'liquid-ooze', 'clear-body']}

In [152]:
def get_pokedata(url):
    """get_pokedata(url):
    Fetch individual pokemon data from pokeapi by url
    Params:
        url: known url of individual pokemon
    Returns:
        Pokeapi response data for individual pokemon
    """
    return requests.get(url).json()

### Pagination

2. Get the same information for the first **151** Pokemon as a list of dictionaries ordered by Pokemon ID. Print the first and last elements of the list. (Hint: Use pagination) Your output should save the list to a variable and look like this:

```
[{'id': 1, 
'name': 'bulbasaur', 
'base_experience': 64, 
'weight': 69, 
'height': 7, 
'types': ['poison', 'grass'], 
'abilities': ['chlorophyll', 'overgrow']}, 
{'id': 2, 
'name': 'ivysaur', 
'base_experience': 142, 
'weight': 130, 
'height': 10, 
'types': ['poison', 'grass'], 
'abilities': ['chlorophyll', 'overgrow']}, ... ]

```



In [144]:
def get_poke_dict_list(num_pokes=151):
    """get_poke_dict_list(num_pokes=151):
    Get a list of pokemon dictionaries using the 
    Pokemon API
    Params:
        num_pokes: number of pokemons to get
        defaulted to 151.
    Assumption:
        Get contiguous range of pokemons by id
    Returns:
        ordered_list is a list of sorted pokemon dicts
    """
    poke_data = []
    
    for i in range(1,num_pokes):
        poke_response = get_pokedata_by_id(i)
        poke_dict = get_pokemon_dict(poke_response)
        poke_data.append(poke_dict)    

    # the list should already be in order, but let's sort it
    # to make sure - sort by pokemon ID
    ordered_list = sorted(poke_data, key=lambda x: x['id'])
    return ordered_list

In [155]:
#
# test get_poke_dict_list
#

In [156]:
pokedata = get_poke_dict_list()

In [158]:
#
# It took a while to get 151 Pokemons!
# So save our poke data so we don't have to re-run all the requests!
#
import json
with open('pokedata.json', 'w') as pokedata_file:
    json.dump(pokedata, pokedata_file)

In [193]:
# printing first and last elements

print(pokedata[0], pokedata[-1])

{'id': 1, 'name': 'bulbasaur', 'base_experience': 64, 'weight': 69, 'height': 7, 'types': ['poison', 'grass'], 'abilities': ['chlorophyll', 'overgrow']} {'id': 150, 'name': 'mewtwo', 'base_experience': 306, 'weight': 1220, 'height': 20, 'types': ['psychic'], 'abilities': ['unnerve', 'pressure']}


## Part 2: Object Oriented Programming

We're going to use the data gathered in the previous section on APIs for this section on Object Oriented Programming to instantiate Pokemon objects and write instance methods.

### Creating a Class

1. Create a class called `Pokemon` with an `__init__` method to instantiate the following attributes:
    - ID
    - Name
    - Base experience
    - Weight
    - Height
    - Types
    - Abilities
    



In [ ]:
# if you were unable to get the data from the API in the right format,
# uncomment the code below to access a JSON file with the list of dictionaries

# with open('data/pokemon.json') as f:  
#     pokelist = json.load(f)

In [187]:
class Pokemon:
    """class Pokemon
    Encapsulate the attributes and behavior of a Pokemon
    """
    def __init__(self, poke_dict):
        """__init__(self, poke_dict):
        instantiate a Pokemon class from a 
        dictionary of pokemon attributes
        """
        self.ID = poke_dict['id']
        self.name = poke_dict['name']
        self.exp = poke_dict['base_experience']
        self.weight = poke_dict['weight']
        self.height = poke_dict['height']
        self.types = poke_dict['types']
        self.abilities = poke_dict['abilities']
        
    def bmi(self):
        """bmi(self):
        BMI is defined by $\frac{weight}{height^{2}}$ with weight 
        in **kilograms** and height in **meters**. 
        The height and weight data of Pokemon from the API is 
        in **decimeters** and **hectograms** respectively.
        
        Returns: bmi
        """
        # convert height:
        conv_height = self.height / 10
        conv_weight = self.weight / 10
        
        return round(conv_weight / conv_height**2, 2)
        


In [188]:
import copy
def get_pokemon_dict_by_name(poke_dict_list, name):
    """get_pokemon_dict_by_name(poke_dict, name):
    Brute force search for individual Pokemon by name
    in given list of pokemon dicts
    Params:
        poke_dict_list: the list of pokemon dictionaries to search
        name: the name of the pokemon to find and fetch
    Returns:
        poke_dict: individual pokemon dict
    """
    poke_dict = {}
    for poke in poke_dict_list:
        if poke['name'] == name:
            # deep copy due to nested structure
            # in case we later manipulate 
            poke_dict = copy.deepcopy(poke)
    
    return poke_dict

    
### Instantiating Objects

2. Using the data you obtained from the API, instantiate the first, fourth and seventh Pokemon. Assign them to the variables `bulbasaur`, `charmander` and `squirtle`.

In [189]:
# use a helper function to fetch pokemon dicts by name
# and instaniate our objects:
bulbasaur = Pokemon(get_pokemon_dict_by_name(pokedata, 'bulbasaur'))
charmander =  Pokemon(get_pokemon_dict_by_name(pokedata, 'charmander'))
squirtle = Pokemon(get_pokemon_dict_by_name(pokedata, 'squirtle'))
ivysaur = Pokemon(get_pokemon_dict_by_name(pokedata, 'ivysaur'))
venusaur = Pokemon(get_pokemon_dict_by_name(pokedata, 'venusaur'))

In [168]:
bulbasaur

In [391]:
# run this cell to test and check your code
# you may need to edit the attribute variable names if you named them differently!

def print_pokeinfo(pokemon_object):
    o = pokemon_object
    print('ID: ' + str(o.ID) + '\n' +
          'Name: ' + o.name.title() + '\n' +
          'Base experience: ' + str(o.exp) + '\n' +
          'Weight: ' + str(o.weight) + '\n' +
          'Height: ' + str(o.height) + '\n' +
          'Types: ' + str(o.types) + '\n' +
          'Abilities: ' + str(o.abilities) + '\n'
         )
    
print_pokeinfo(bulbasaur)
print_pokeinfo(ivysaur)
print_pokeinfo(venusaur)

ID: 1
Name: Bulbasaur
Base experience: 64
Weight: 69
Height: 7
Types: ['poison', 'grass']
Abilities: ['chlorophyll', 'overgrow']

ID: 2
Name: Ivysaur
Base experience: 142
Weight: 130
Height: 10
Types: ['poison', 'grass']
Abilities: ['chlorophyll', 'overgrow']

ID: 3
Name: Venusaur
Base experience: 236
Weight: 1000
Height: 20
Types: ['poison', 'grass']
Abilities: ['chlorophyll', 'overgrow']



### Instance Methods

3. Write an instance method within the class `Pokemon` to find the BMI of a Pokemon. BMI is defined by $\frac{weight}{height^{2}}$ with weight in **kilograms** and height in **meters**. The height and weight data of Pokemon from the API is in **decimeters** and **hectograms** respectively.


    1 decimeter = 0.1 meters
    1 hectogram = 0.1 kilograms

In [392]:
# run this cell to test and check your code
# you will probably have to rerun the code to instantiate your objects

print(bulbasaur.bmi()) # 14.08
print(charmander.bmi()) # 23.61
print(squirtle.bmi()) # 36

14.08
23.61
36.0


## Part 3: SQL and Relational Databases

For this section, we've put the Pokemon data into SQL tables. You won't need to use your list of dictionaries or the JSON file for this section. The schema of `pokemon.db` is as follows:

<img src="data/pokemondb.png" alt="db schema" style="width:500px;"/>

Assign your SQL queries as strings to the variables `q1`, `q2`, etc. and run the cells at the end of this section to print your results as Pandas DataFrames.

- q1: query all columns from `Pokemon` the Pokemon that have base_experience above 200  

  
- q2: query the id, name, type1 and type2 of Pokemon that have **water** types as either their first or second type


- q3: query the average weight of Pokemon by their first type in descending order


- q4: query the Pokemon name, Pokemon type2, and what **type2** has "2xdamage" to


- q5: query the top 5 most common type1s, the minimum height, maximum height, minimum weight and maximum weight of pokemon with those type1s, and what associated type they do "0.5xdamage" to


**Important note on syntax**: use `double quotes ""` when quoting strings **within** your query and wrap the entire query in `single quotes ''` For the column titles that begin with numbers, you need to wrap the column names in double quotes.

In [393]:
xb1 = Pokemon(b)

In [394]:
cnx = sqlite3.connect('data/pokemon.db')

In [395]:
q1 = """select * from pokemon
where base_experience > 200
"""
pd.read_sql(q1, cnx)

,id,name,base_experience,weight,height,type1,type2
0,3,venusaur,236,1000,20,grass,poison
1,6,charizard,240,905,17,fire,flying
2,9,blastoise,239,855,16,water,None
3,18,pidgeot,216,395,15,normal,flying
4,26,raichu,218,300,8,electric,None
5,31,nidoqueen,227,600,13,poison,ground
6,34,nidoking,227,620,14,poison,ground
7,36,clefable,217,400,13,fairy,None
8,45,vileplume,221,186,12,grass,poison
9,62,poliwrath,230,540,13,water,fighting


In [198]:
q2 = """select id, name, type1, type2
from pokemon
where type1 = 'water' or type2 = 'water'
"""
pd.read_sql(q2, cnx)

,id,name,type1,type2
0,7,squirtle,water,None
1,8,wartortle,water,None
2,9,blastoise,water,None
3,54,psyduck,water,None
4,55,golduck,water,None
5,60,poliwag,water,None
6,61,poliwhirl,water,None
7,62,poliwrath,water,fighting
8,72,tentacool,water,poison
9,73,tentacruel,water,poison


In [396]:
q3 = """select type1, round(avg(weight), 2) as average_weight
from pokemon
group by type1
"""
pd.read_sql(q3, cnx)

,type1,average_weight
0,bug,229.92
1,dragon,766.00
2,electric,317.89
3,fairy,237.50
4,fighting,542.86
5,fire,480.25
6,ghost,135.67
7,grass,279.92
8,ground,452.63
9,ice,480.00


In [200]:
q4 = """select p.name, p.type2, t."2xdamage"
from pokemon p
join types t
using(id)
"""
pd.read_sql(q4, cnx)

,name,type2,2xdamage
0,bulbasaur,poison,None
1,ivysaur,poison,normal
2,venusaur,poison,fighting
3,charmander,None,grass
4,charmeleon,None,poison
5,charizard,flying,flying
6,squirtle,None,grass
7,wartortle,None,ghost
8,blastoise,None,rock
9,caterpie,None,bug


In [397]:
q5 = """select min(height), max(height), min(weight), max(weight), t.name, t."0.5xdamage"
from pokemon as p
join
(select type1, count(type1) as cnt
from pokemon
group by type1
order by cnt desc
limit 5) as c
using(type1)
join types t
on p.id = t.id"""
pd.read_sql(q5, cnx)

,min(height),max(height),min(weight),max(weight),name,0.5xdamage
0,3,17,18,905,rock,normal


## Section 4: Web Scraping

### Accessing Data Using BeautifulSoup

Use BeautifulSoup to get quotes, authors, and tags from [Quotes to Read](http://quotes.toscrape.com/).

Before answering these questions, go to the site and inspect the page. Make sure to look at what links there are and how the site is structured.

1. Get the first author and the path for the author's page as a tuple from the [homepage](http://quotes.toscrape.com/).

In [398]:
# Make a get request to retrieve the page
html_page = requests.get('http://quotes.toscrape.com/') 
# Pass the page contents to beautiful soup for parsing
soup = BeautifulSoup(html_page.content, 'html.parser')

# define a func to pull out author name and href from
# specific quote section of soup
#
def get_author_and_href(soup):
    """get_author_and_href(soup):
    Given some soup which points to a SINGLE
    quote, fetch the author name
    and author href
    Params:
        soup: soup for SINGLE quote
    Returns:
        (auth, auth_href)
    """
    auth = soup.find('small', class_='author').contents[0]
    auth_href = soup.find('a').attrs['href']
    
    return (auth, auth_href)

#
# test our func:
# 1. get all quotes
# 2. pass first quote to our func
# 
quotes = soup.findAll('div', class_='quote')
print(get_author_and_href(quotes[0]))

('Albert Einstein', '/author/Albert-Einstein')


In [231]:
get_author_and_href(quotes[0])

('Albert Einstein', '/author/Albert-Einstein')

In [399]:
def get_quotes(soup):
    """get_quote_soup(url):
    Get the quotes section of the response
    Params:
        url:  soup that contains response
    Returns:
        quotes = soup containing quotes only
    """
    return soup.findAll('div', class_='quote')

In [400]:
def get_soup(url):
    """get_soup(url):
    Get the soup from the given url
    Params:
        url:  url to use
    Returns:
        soup:  Beautiful Soup object
    """
    html_page = requests.get(url) 

    return BeautifulSoup(html_page.content, 'html.parser')

In [401]:
def get_next_url(soup):
    """get_next_url(soup):
    Get the next url link from our
    main soup BEFORE pulling out quotes
    
    """
    n = soup.find('li', class_='next')
    
    return n.find('a').attrs['href']

2. Write a function to get **all** the authors and href links for the authors from the [homepage](http://quotes.toscrape.com/)


In [402]:
def get_authors(quotes):
    """
    input: quotes that have already been extracted
    from soup
    
    return: a dictionary of of authors and their urls
            {'author_1':'url_of_author_1', 'author_2':'url_of_author_2' ...}
    """
    author_dict = {}

    for quote in quotes:
        auth, href = get_author_and_href(quote)
        author_dict[auth] = href
        
    return author_dict

In [403]:
# run this cell to test the function
#
# THIS TEST IS RE-WRITTEN TO USE OUR RE-FACTORED FUNCTIONS!!
#
url = 'http://quotes.toscrape.com/'
soup = get_soup(url)
quotes = get_quotes(soup)
print(get_authors(quotes))

print('\n')

url = 'http://quotes.toscrape.com/page/3'
soup = get_soup(url)
quotes = get_quotes(soup)
print(get_authors(quotes))


{'Albert Einstein': '/author/Albert-Einstein', 'J.K. Rowling': '/author/J-K-Rowling', 'Jane Austen': '/author/Jane-Austen', 'Marilyn Monroe': '/author/Marilyn-Monroe', 'André Gide': '/author/Andre-Gide', 'Thomas A. Edison': '/author/Thomas-A-Edison', 'Eleanor Roosevelt': '/author/Eleanor-Roosevelt', 'Steve Martin': '/author/Steve-Martin'}


{'Pablo Neruda': '/author/Pablo-Neruda', 'Ralph Waldo Emerson': '/author/Ralph-Waldo-Emerson', 'Mother Teresa': '/author/Mother-Teresa', 'Garrison Keillor': '/author/Garrison-Keillor', 'Jim Henson': '/author/Jim-Henson', 'Dr. Seuss': '/author/Dr-Seuss', 'Albert Einstein': '/author/Albert-Einstein', 'J.K. Rowling': '/author/J-K-Rowling', 'Bob Marley': '/author/Bob-Marley'}


### Pagination

3. Get the first author on each of the first 5 pages of quotes. You can get to the next page with the next button at the bottom of the homepage.


In [333]:
def get_next_url(soup):
    """get_next_url(soup):
    Get the next url link from our
    main soup BEFORE pulling out quotes
    
    """
    n = soup.find('li', class_='next')
    return n.find('a').attrs['href']

In [404]:
BASE_URL = 'http://quotes.toscrape.com'
nxt_page = ''
auth_list = []

# We are explicitly only getting 
# pages 1-5!
for page in range(1,6):
    url = BASE_URL + nxt_page    

    # get soup for url:
    soup = get_soup(url)

    # get next page:
    nxt_page = get_next_url(soup)

    # get quotes
    quotes = get_quote_soup(soup)
    
    # couple of ways to do this:  
    # let's simply pull out the author 
    # of the FIRST quote (quote[0]).
    # Use "_" for href portion of tuple since
    # we ignore it!
    auth, _ = get_author_and_href(quotes[0])

    auth_list.append(auth)

In [405]:
auth_list

['Albert Einstein',
 'Marilyn Monroe',
 'Pablo Neruda',
 'Dr. Seuss',
 'George R.R. Martin']

4. Write a function to get all of the quotes from a page.

In [388]:
def get_some_quotes(url, num_pages=1):
    '''
    input: url, number of pages to scrape (just scrape the home page if no argument is passed in)
    
    return: a list of dictionaries of quotes with their attributes
            [{'quote':'quote_1_text', 'author':'url_of_author_1'}, 
            {'quote':'quote_2_text', 'author':'url_of_author_2', 'quote_tags':[list_of_quote_2_tags]}, ...]
    '''
    nxt_page = ''
    
    quote_list = []
    
    # ensure we get at least one page, so start at 0;
    # for each page we fetch the "next" link and save it
    #
    # Function assumes we start with the "homepage",
    # but caller may pass in url of "page 2", which would
    # break.
    # TO DO:  check url argument first and pull out 
    # "base" url 
    #
    for page in range(0, num_pages):
        url = url + nxt_page    
        soup = get_soup(url)
        nxt = get_next_url(soup)
        quotes = get_quotes(soup)

        for quote in quotes:
            auth, href = get_author_and_href(quote)
            quote_text = quote.find('span', class_='text').contents[0]
            quote_dict = {'quote': quote_text, auth : href}
            quote_list.append(quote_dict)

    return quote_list
    

In [406]:
quotes = get_some_quotes('http://quotes.toscrape.com', 2)

In [407]:
quotes

[{'quote': '“The world as we have created it is a process of our thinking. It cannot be changed without changing our thinking.”',
  'Albert Einstein': '/author/Albert-Einstein'},
 {'quote': '“It is our choices, Harry, that show what we truly are, far more than our abilities.”',
  'J.K. Rowling': '/author/J-K-Rowling'},
 {'quote': '“There are only two ways to live your life. One is as though nothing is a miracle. The other is as though everything is a miracle.”',
  'Albert Einstein': '/author/Albert-Einstein'},
 {'quote': '“The person, be it gentleman or lady, who has not pleasure in a good novel, must be intolerably stupid.”',
  'Jane Austen': '/author/Jane-Austen'},
 {'quote': "“Imperfection is beauty, madness is genius and it's better to be absolutely ridiculous than absolutely boring.”",
  'Marilyn Monroe': '/author/Marilyn-Monroe'},
 {'quote': '“Try not to become a man of success. Rather become a man of value.”',
  'Albert Einstein': '/author/Albert-Einstein'},
 {'quote': '“It is b

In [408]:
# set the function to a variable to use later
quotes_for_mongo = get_some_quotes('http://quotes.toscrape.com/' )
quotes_for_mongo

[{'quote': '“The world as we have created it is a process of our thinking. It cannot be changed without changing our thinking.”',
  'Albert Einstein': '/author/Albert-Einstein'},
 {'quote': '“It is our choices, Harry, that show what we truly are, far more than our abilities.”',
  'J.K. Rowling': '/author/J-K-Rowling'},
 {'quote': '“There are only two ways to live your life. One is as though nothing is a miracle. The other is as though everything is a miracle.”',
  'Albert Einstein': '/author/Albert-Einstein'},
 {'quote': '“The person, be it gentleman or lady, who has not pleasure in a good novel, must be intolerably stupid.”',
  'Jane Austen': '/author/Jane-Austen'},
 {'quote': "“Imperfection is beauty, madness is genius and it's better to be absolutely ridiculous than absolutely boring.”",
  'Marilyn Monroe': '/author/Marilyn-Monroe'},
 {'quote': '“Try not to become a man of success. Rather become a man of value.”',
  'Albert Einstein': '/author/Albert-Einstein'},
 {'quote': '“It is b

## Part 5: MongoDB

To do this section, open a connection to a mongo database in the terminal, using `mongod`. You will then **create**, **update**, and **read** from a mongo database.

Create and connect to a mongo database.

In [ ]:
myclient = pymongo.MongoClient("mongodb://127.0.0.1:27017/")
mydb = myclient['quote_database']

In [ ]:
mycollection = mydb['quote_collection']

1. Add the quotes you obtained from the `get_some_quotes` function for the [homepage](http://quotes.toscrape.com/) to the mongo database. (You can also use the JSON file `quotes.json` to insert data into the database) To verify that you've successfully inserted the data, query it to obtain the resulting _ids back from the `results` variable. 

In [ ]:
# if you were unable to get the data from webscraping in the right format,
# uncomment the code below to access a JSON file with the list of dictionaries

# with open(r"data/quotes.json", "r") as r:
#     data = json.load(r)

In [ ]:
# use the results variable to confirm the data was inserted
results = None

2. Query the database for all the quotes written by `'Albert Einstein'`.

In [ ]:
q1 = None

3. Update Steve Martin's quote with the tags for the quote stored in the variable `steve_martin_tags`.

In [ ]:
steve_martin_tags = {'quote_tags': ['change', 'deep-thoughts', 'thinking', 'world']}
update_steve = None
first_quote_tags = None


4. Query the database to confirm that  `'Steve Martin'` is updated with `steve_martin_tags`.

In [ ]:
q2 = None